In [ ]:
!pip install tqdm

# Step 1: Downloading the dataset

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-08-12 08:56:05--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 173.194.76.128, 64.233.184.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  56.7MB/s    in 1.2s    

2020-08-12 08:56:06 (56.7 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



# Step 2: Data preprocessing

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

## Unzipping the zip file

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [ ]:
zip_object.extractall('./')

In [ ]:
zip_object.close()

## Setting up the dataset

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

# Step 3: Building the model

## Loading the Pre-trained model(MobileNetV2)

In [ ]:
IMG_shape = (128,128,3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_shape, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

## freezing the base model
otherwise it will change the initial weights

In [ ]:
base_model.trainable = False

## Defining the custom head for the network

In [ ]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

## Concatenating 2 networks(defining the exact model)

In [ ]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

## Compiling the model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

## Creating the data generator
resizing the images



Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = data_gen_train.flow_from_directory(directory=train_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [ ]:
valid_generator = data_gen_valid.flow_from_directory(directory=validation_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 1000 images belonging to 2 classes.


## Training the model

In [ ]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
16/16 [==============================] - 11s 657ms/step - loss: 0.7879 - accuracy: 0.5355 - val_loss: 0.7547 - val_accuracy: 0.5630
Epoch 2/10
16/16 [==============================] - 9s 557ms/step - loss: 0.6637 - accuracy: 0.6350 - val_loss: 0.6545 - val_accuracy: 0.6360
Epoch 3/10
16/16 [==============================] - 9s 552ms/step - loss: 0.5774 - accuracy: 0.7145 - val_loss: 0.5691 - val_accuracy: 0.7020
Epoch 4/10
16/16 [==============================] - 9s 553ms/step - loss: 0.5037 - accuracy: 0.7780 - val_loss: 0.4997 - val_accuracy: 0.7590
Epoch 5/10
16/16 [==============================] - 9s 557ms/step - loss: 0.4422 - accuracy: 0.8170 - val_loss: 0.4392 - val_accuracy: 0.8010
Epoch 6/10
16/16 [==============================] - 9s 553ms/step - loss: 0.3907 - accuracy: 0.8560 - val_loss: 0.3889 - val_accuracy: 0.8340
Epoch 7/10
16/16 [==============================] - 9s 554ms/step - los

## model evaluation (transfer learning process)

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.906000018119812


# Step 4: Fine Tunning

## Unfreeze the layers from the model

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


In [ ]:
fine_tune_at = 100
for layers in base_model.layers[:fine_tune_at]:
  base_model.trainable = False

## Compiling the fine tunning model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

## fitting the fine tunning model


In [ ]:
model.fit(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
16/16 [==============================] - 10s 635ms/step - loss: 0.2304 - accuracy: 0.9275 - val_loss: 0.2270 - val_accuracy: 0.9230
Epoch 2/10
16/16 [==============================] - 9s 558ms/step - loss: 0.2109 - accuracy: 0.9305 - val_loss: 0.2106 - val_accuracy: 0.9290
Epoch 3/10
16/16 [==============================] - 9s 566ms/step - loss: 0.1973 - accuracy: 0.9315 - val_loss: 0.1969 - val_accuracy: 0.9360
Epoch 4/10
16/16 [==============================] - 9s 563ms/step - loss: 0.1860 - accuracy: 0.9355 - val_loss: 0.1853 - val_accuracy: 0.9410
Epoch 5/10
16/16 [==============================] - 9s 564ms/step - loss: 0.1764 - accuracy: 0.9380 - val_loss: 0.1756 - val_accuracy: 0.9420
Epoch 6/10
16/16 [==============================] - 9s 569ms/step - loss: 0.1682 - accuracy: 0.9400 - val_loss: 0.1673 - val_accuracy: 0.9450
Epoch 7/10
16/16 [==============================] - 9s 560ms/step - loss: 0.1613 - accuracy: 0.9435 - val_loss: 0.1606 - val_accuracy: 0.9460
Epoch

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 3s 336ms/step - loss: 0.1449 - accuracy: 0.9490


In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9490000009536743
